In [166]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
from src.poly_reg import model_county
from src.poly_reg import model_state
from src.data_import import nyt
import src.data_import as di

plt.style.use('ggplot')
font = {'weight' : 'bold',
        'size'   : 18}

plt.rc('font', **font)

In [32]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

df = di.extract_election()

In [33]:
cadf = df[df['state'] == 'California']

In [34]:
len(cadf)

58

In [154]:
#counties_df = pd.read_csv("./data/raw/County Physical.csv", sep='\t')
counties_df = di.extract_geography()

In [162]:
counties_df[counties_df['State'] == 'California']
counties_df


,Sort,State,FIPS,County,Seat,Pop,Land area km,Land area mi,Water area km,Water area mi,Total area km,Total area mi,Lat,Lon,sc
0,1,Alabama,1001,Autauga,Prattville,54571,1539.582,594.436,25.776,9.952,1565.358,604.388,32.536382,-86.644490,Alabama:Autauga
1,2,Alabama,1003,Baldwin,Bay Minette,182265,4117.522,1589.784,1133.190,437.527,5250.712,2027.311,30.659218,-87.746067,Alabama:Baldwin
2,3,Alabama,1005,Barbour,Clayton,27457,2291.819,884.876,50.865,19.639,2342.684,904.515,31.870670,-85.405456,Alabama:Barbour
3,4,Alabama,1007,Bibb,Centreville,22915,1612.481,622.582,9.289,3.587,1621.770,626.169,33.015893,-87.127148,Alabama:Bibb
4,5,Alabama,1009,Blount,Oneonta,57322,1669.962,644.776,15.157,5.852,1685.119,650.628,33.977448,-86.567246,Alabama:Blount
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,3139,Wyoming,56037,Sweetwater,Green River,43806,27004.897,10426.649,166.887,64.436,27171.784,10491.085,41.660339,-108.875676,Wyoming:Sweetwater
3139,3140,Wyoming,56039,Teton,Jackson,21294,10347.984,3995.379,572.266,220.953,10920.250,4216.332,44.049321,-110.588102,Wyoming:Teton
3140,3141,Wyoming,56041,Uinta,Evanston,21118,5390.450,2081.264,16.342,6.310,5406.791,2087.574,41.284726,-110.558947,Wyoming:Uinta
3141,3142,Wyoming,56043,Washakie,Worland,8533,5797.815,2238.549,10.762,4.155,5808.577,2242.704,43.878831,-107.669052,Wyoming:Washakie


In [152]:
def convert_numbers(ap):
    ap.pax = int(ap.pax.replace(',', ''))
    ap.domestic = int(ap.domestic.replace(',', ''))
    ap.international = int(ap.international.replace(',', ''))
    return ap

airports_df = di.extract_airports()
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   airport        50 non-null     object 
 1   pax            50 non-null     int64  
 2   domestic       50 non-null     int64  
 3   international  50 non-null     int64  
 4   lat            50 non-null     float64
 5   lon            50 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 2.5+ KB


In [168]:
import mpu


def calc_intl_arrivals_index(lat, lon, threshold, airports_df):
    domestic = 0
    intl = 0
    airports = []
    for i, airport in airports_df.iterrows():
        #print(airport)
        alat = airport['lat']
        alon = airport['lon']
        dist = mpu.haversine_distance((lat, lon), (alat, alon))
        #print('{} is {} km away'.format(airport['airport'], dist))
        if dist < threshold:
            intl += int(airport['international'])
            domestic += int(airport['domestic'])
            airports.append(airport['airport'])
    return intl, domestic, airports
        
#intl_arrival_index(37.648081, -121.913304)
#intl_arrival_index(37.7749,-122.4194 , 100, airports_df)
def append_intl_arrivals_index(counties_df, airports_df, threshold):
    res = pd.DataFrame(columns=['sc', 'international', 'domestic', 'airports'])
    for i, county in counties_df.iterrows():
        intl, domestic, airports = calc_intl_arrivals_index(county['Lat'], float(county['Lon']), threshold, airports_df)
        res.loc[i] = [county['sc'], intl, domestic, airports]
    return res

x = di.build_intl_arrivals_index_df(counties_df, airports_df, 100)
x


,sc,international,domestic,airports
0,Alabama:Autauga,0,0,[]
1,Alabama:Baldwin,0,0,[]
2,Alabama:Barbour,0,0,[]
3,Alabama:Bibb,0,0,[]
4,Alabama:Blount,0,0,[]
...,...,...,...,...
3138,Wyoming:Sweetwater,0,0,[]
3139,Wyoming:Teton,0,0,[]
3140,Wyoming:Uinta,0,0,[]
3141,Wyoming:Washakie,0,0,[]


In [169]:
x[x['international'] >0]

,sc,international,domestic,airports
103,Arizona:Maricopa,1022695,20574818,"[Phoenix, AZ (PHX)]"
107,Arizona:Pinal,1022695,20574818,"[Phoenix, AZ (PHX)]"
186,California:Alameda,7335814,32560804,"[San Francisco, CA (SFO), Oakland, CA (OAK), S..."
188,California:Amador,112084,5343355,"[Sacramento, CA (SMF)]"
191,California:Colusa,112084,5343355,"[Sacramento, CA (SMF)]"
...,...,...,...,...
3103,Wisconsin:St. Croix,1453112,17025649,"[Minneapolis, MN (MSP)]"
3107,Wisconsin:Sheboygan,66751,3421109,"[Milwaukee, WI (MKE)]"
3112,Wisconsin:Walworth,6463618,35747736,"[Chicago, IL (ORD), Milwaukee, WI (MKE)]"
3114,Wisconsin:Washington,66751,3421109,"[Milwaukee, WI (MKE)]"
